In [1]:
import numpy as np
import cv2
from scipy.spatial import distance as dist
from matplotlib import pyplot as plt
import math
from scipy.optimize import curve_fit 
%matplotlib inline 
%matplotlib qt

In [2]:
min_thresh = 120
def nothing(x):
    pass

def blur(img,kernal,value,iterations):
    imgblur = cv2.GaussianBlur(img,kernal,value)
    for i in range(iterations-1):
        imgblur = cv2.GaussianBlur(imgblur,kernal,value)
    return imgblur

def DrawBox(img,cnt):
    rect = cv2.minAreaRect(cnt)
    box  = cv2.boxPoints(rect)
    box  = np.int0(box)
    return cv2.drawContours(img,[box],0,(134,112,255),2),rect

def DrawOval(img,cnt):
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    center = (int(x),int(y))
    radius = int(radius)
    return cv2.circle(img,center,radius,(0,255,0),2)

def order_points(pts):
        xSorted = pts[np.argsort(pts[:, 0]), :]

        leftMost = xSorted[:2, :]
        rightMost = xSorted[2:, :]

        leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
        (tl, bl) = leftMost

        D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
        (br, tr) = rightMost[np.argsort(D)[::-1], :]
        
        return np.asarray([tl, tr, br, bl], dtype=pts.dtype)
    
def crop_rotated_rectangle(img,rect):
        rbox = order_points(cv2.boxPoints(rect))
        
        # get width and height of the detected rectangle
        
        width   = np.linalg.norm([rbox[0, 0] - rbox[1, 0], rbox[0, 1] - rbox[1, 1]])
        height  = np.linalg.norm([rbox[0, 0] - rbox[-1, 0], rbox[0, 1] - rbox[-1, 1]])
        src_pts = rbox.astype(np.float32)
        
        # coordinate of the points in box points after the rectangle has been straightened
        # this step needs order_points to be called on src
        dst_pts = np.array([[0, 0],
                            [width - 1, 0],
                            [width - 1, height - 1],
                            [0, height - 1]], dtype="float32")
        # the perspective transformation matrix
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        # directly warp the rotated rectangle to get the straightened rectangle
        warped = cv2.warpPerspective(img, M, (width, height), None, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT,
                                     (255, 255, 255))
        return warped 
    
def brightness_contrast(image,alpha,beta):
    
    #temp = image.copy()
    #temp.convertTo(temp, -1, 1, -128);
    
    temp_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    #new_image = np.zeros(image.shape, image.dtype)
    # Do the operation new_image(i,j) = alpha*image(i,j) + beta
    # Instead of these 'for' loops we could have used simply:
    #temp.convertTo(new_image, alpha=alpha, beta=beta+128)
    # but we wanted to show you how to access the pixels :)
    
    #for y in range(image.shape[0]):
        #for x in range(image.shape[1]):
            #new_image[y,x] = alpha*(image[y,x]-128)+ 128 + beta
                
    return temp_image

def spiral_equation(originx,originy,k,xs,a,ys,max_theta):
    theta = np.arange(0,max_theta,0.1)
    #xs = 100
    #ax = 0
    #k = 1
    
    #ys = 100
    #ay = 0
    
    x = np.array([])
    y = np.array([])
  
    for i in range(len(theta)): 
        x=np.append(x,originx+theta[i]*xs*math.cos(a+k*theta[i]))
        y=np.append(y,originy+theta[i]*ys*math.sin(a+k*theta[i]))
        i += 1
    
    xy = np.column_stack((x, y))
    xy = xy.reshape((-1,1,2))
    return np.int0(xy)   

In [3]:
GALAXY_DETECT_THRESH  = 40
CORE_DETECT_THRESH    = 230
ERODE_KERNAL          = np.ones((5,5),np.uint8)

def contour_detection(image,threshold,ed_iterations):
    imageblur = blur(image,(5,5),0,4)
    
    ret, threshold_img = cv2.threshold(imageblur, threshold, 255, cv2.THRESH_BINARY)
    
    threshold_img = cv2.erode(threshold_img,ERODE_KERNAL,iterations = ed_iterations)
    threshold_img = cv2.dilate(threshold_img,ERODE_KERNAL,iterations = ed_iterations)
    
    contour_img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    _, contours, hierarchy = cv2.findContours(threshold_img.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contour_img            = cv2.drawContours(contour_img, contours, -1, (121,255,134), 3)
    return threshold_img,contour_img,contours

def detect_orientation(galaxy, threshold,ed_iterations):
    detectedgal,contour, contours = contour_detection(galaxy,threshold,ed_iterations)

    maxsize=0
    for i in range(len(contours)):
        if cv2.contourArea(contours[i])>maxsize:
            cnt=contours[i]
            maxsize=cv2.contourArea(cnt)
   
    try:
        contour,rect = DrawBox(contour,cnt)
        return detectedgal, contour,rect
    except:
        return detectedgal, galaxy, None 
        print("")

def filter_out(image,min_thresh,max_thresh):
    temp=cv2.inRange(image,min_thresh,max_thresh)
    nottemp = cv2.bitwise_not(temp)
    return cv2.bitwise_and(nottemp,image)

def remove_stars(image,threshold,ed_iterations):
    
    contour_img = image.copy()
    ___,____, contours = contour_detection(image,threshold,ed_iterations)
    
    mask = np.ones(image.shape[:2], dtype="uint8") * 255
    
    maxsize=0
    for i in range(len(contours)):
        if cv2.contourArea(contours[i])>maxsize:
            cnt=contours[i]
            maxsize=cv2.contourArea(cnt)
            
    for c in contours:
        if cv2.contourArea(c) != maxsize:
            cv2.drawContours(mask, [c], -1, 0, -1)
            
    contour_img = cv2.bitwise_and(contour_img, contour_img, mask=mask)
    
    return contour_img

def approx_contour(image,cnt,alpha): 
    contour_img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    epsilon     = alpha*cv2.arcLength(cnt,True)
    approx      = cv2.approxPolyDP(cnt,epsilon,True)
    #print(approx)
    contour_img = cv2.drawContours(contour_img, [approx], -1, (121,255,134), 3)
    return contour_img,approx

In [4]:
def spiral_equation(originx,originy,k,xs,a,ys,max_theta):
    theta = np.arange(0,max_theta,0.1)
    #xs = 100
    #ax = 0
    #k = 1
    
    #ys = 100
    #ay = 0
    
    x = np.array([])
    y = np.array([])
  
    for i in range(len(theta)): 
        x=np.append(x,originx+theta[i]*xs*math.cos(a+k*theta[i]))
        y=np.append(y,originy+theta[i]*ys*math.sin(a+k*theta[i]))
        i += 1
    
    xy = np.column_stack((x, y))
    xy = xy.reshape((-1,1,2))
    return np.int0(xy)   

#def identify lines():
    
    

In [5]:
ngc1300=cv2.imread('ngc-1300.jpg',0)
m101hires=cv2.imread('M101-hires.jpg',0)
ngc2997=cv2.imread('ngc-2997.jpg',0)
ngc4030=cv2.imread('ngc-4030.jpg',0)
ngc4414=cv2.imread('ngc-4414.jpg',0)
ngc5054=cv2.imread('ngc-5054.jpg',0)
ngc5247=cv2.imread('ngc-5247.jpg',0)
galaxy = ngc5054.copy()

In [8]:
#cv2.createTrackbar('min_tresh','Threshold',220,255,nothing) #40

#while(1):
#kernel_sharpening = np.array([[-1,-1,-1], 
#                              [-1, 9,-1],
#                              [-1,-1,-1]])
DOING_ARMS    = False
DOING_EDGES   = False
DOING_CONTOUR = False
DOING_SPIRAL  = False 
#Detect orientation and correct it
detectedgal, contour,rect = detect_orientation(galaxy,GALAXY_DETECT_THRESH,5)

#blacken out rest of pic
exact_gal = cv2.bitwise_and(detectedgal,galaxy)

#extract the galaxy
extracted_gal = crop_rotated_rectangle(exact_gal,rect) 

#extract the core
detectedcore, corecontour,_ = detect_orientation(extracted_gal,CORE_DETECT_THRESH,5)

alpha= 122  #cv2.getTrackbarPos('alpha','Brightness Contrast')
beta = 48   #cv2.getTrackbarPos('beta','Brightness Contrast')

#Lower brightness and increase contrast
contrasted_gal = brightness_contrast(extracted_gal,(alpha)/100,beta-127)
contrasted_gal = filter_out(contrasted_gal,79,79)

#Without shiny stars
wo_stars       = remove_stars(contrasted_gal,67,1)

    
wo_stars = cv2.erode(wo_stars,ERODE_KERNAL,iterations = 1)
wo_stars = cv2.dilate(wo_stars,ERODE_KERNAL,iterations = 1)


#cv2.namedWindow('Detected Galaxy',cv2.WINDOW_NORMAL)

if DOING_EDGES:
    cv2.namedWindow('Edges',cv2.WINDOW_NORMAL)
    cv2.createTrackbar('max_tresh','Edges',0,255,nothing) #40
    
if DOING_ARMS:
    cv2.namedWindow('Arms',cv2.WINDOW_NORMAL)
    cv2.createTrackbar('min_tresh','Arms',27,255,nothing) 
    cv2.createTrackbar('max_tresh','Arms',10,255,nothing)
    
if DOING_CONTOUR:
    cv2.namedWindow('Contours',cv2.WINDOW_NORMAL)
    cv2.namedWindow('Contours1',cv2.WINDOW_NORMAL)
    cv2.createTrackbar('min_tresh','Contours',53,255,nothing)
    cv2.createTrackbar('min_tresh','Contours1',6,10,nothing)
    
if DOING_SPIRAL:
    cv2.namedWindow('Spirals',cv2.WINDOW_NORMAL)
    cv2.createTrackbar('k' ,'Spirals',125,200,nothing)
    cv2.createTrackbar('a' ,'Spirals',0  ,614,nothing)
    cv2.createTrackbar('xs','Spirals',100,200,nothing)
    cv2.createTrackbar('ys','Spirals',100,200,nothing) 
    
cv2.namedWindow('Brightness Contrast',cv2.WINDOW_NORMAL)
cv2.namedWindow('Extracted Galaxy',cv2.WINDOW_NORMAL)
cv2.namedWindow('Original Image',cv2.WINDOW_NORMAL)
cv2.namedWindow('Temporary',cv2.WINDOW_NORMAL)

#cv2.createTrackbar('alpha','Brightness Contrast',122,300,nothing) #40
#cv2.createTrackbar('beta','Brightness Contrast',48,255,nothing) #40
#cv2.createTrackbar('kernal','Image Sharpening',2,29,nothing)
#plt.ion()

while(1):
      
    if DOING_ARMS:
        arm_min_tresh   = cv2.getTrackbarPos('min_tresh','Arms')
        arm_max_tresh   = cv2.getTrackbarPos('max_tresh','Arms')
        arm_img = cv2.inRange(wo_stars, arm_min_tresh, arm_min_tresh+arm_max_tresh)
        cv2.imshow('Arms',arm_img)
        
    if DOING_EDGES:
        edge_tresh  = cv2.getTrackbarPos('max_tresh','Edges')
        blu = blur(arm_img,(3,3),0,4)
        edges = cv2.Canny(blu,40,edge_tresh)
        cv2.imshow('Edges',edges)
     
    if DOING_SPIRAL:
        k           = cv2.getTrackbarPos('k','Spirals')
        a           = cv2.getTrackbarPos('a','Spirals')
        xs          = cv2.getTrackbarPos('xs','Spirals')
        ys          = cv2.getTrackbarPos('ys','Spirals')
        spiral = cv2.cvtColor(arm_img, cv2.COLOR_GRAY2BGR)
        pts = spiral_equation(np.int0(spiral.shape[0]/2)-200,np.int0(spiral.shape[0]/2),(k-100)/25,xs,a/100,ys,5)
        cv2.polylines(spiral, [pts], False, (0,255,255), 3)
        cv2.imshow('Spirals',spiral)
    
    if DOING_CONTOUR:
        contour_tresh  = cv2.getTrackbarPos('min_tresh','Contours')
        contour_tresh1 = cv2.getTrackbarPos('min_tresh','Contours1')
        detectedcontour, contour_img, contours = contour_detection(arm_img,contour_tresh,1)
        
        maxsize=0
        for i in range(len(contours)):
            if cv2.contourArea(contours[i])>maxsize:
                cnt=contours[i]
                maxsize=cv2.contourArea(cnt)
        contour_img1 ,lines = approx_contour(arm_img,cnt,contour_tresh1/1000)
        #print(lines)
        
        
        cv2.imshow('Contours',contour_img)
        cv2.imshow('Contours1',contour_img1)
    
    #fitcurve(arm_img)
    
    cv2.imshow('Original Image',galaxy)
    cv2.imshow('Extracted Galaxy',extracted_gal)
    cv2.imshow('Brightness Contrast',contrasted_gal)
    cv2.imshow('Temporary',wo_stars)
    cv2.imwrite("Galaxy.jpg",contrasted_gal)
    
    k = cv2.waitKey(100)
    if k == 27:         # wait for ESC key to exit
        break
        #cv2.destroyAllWindows()
        
cv2.destroyAllWindows()